In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from rdkit import Chem
from sklearn.preprocessing import OneHotEncoder

In [21]:
data_1 = pd.read_csv('Data/200d_-1_named+fda.csv')
data_35 = pd.read_csv('Data/200d_3.5_+in-vitro.csv')

print('len(data_1)=',len(data_1))
print('len(data_35)=',len(data_35))

len(data_1)= 1468
len(data_35)= 2964


In [22]:
data_35 = data_35[:len(data_1)]

In [14]:
'''data = data_35
i = 0

mol = Chem.MolFromSmiles(data['smiles'][i]) # transforming the SMILE sequence to Chem object
x = np.array([atom.GetAtomicNum() for atom in mol.GetAtoms()]) # getting the features (atoms number) [1D]
A = Chem.rdmolops.GetAdjacencyMatrix(mol) # extracting the adjecency matrix
edge_index = np.argwhere(A != 0).T # getting the edge_index matrix (2 X edge_num) from the adjecency [2D]'''

In [27]:
def extract_graph_data(data):
    
    data_x = []
    data_edge_index = []

    for i in range(len(data)):
        mol = Chem.MolFromSmiles(data['smiles'][i]) # transforming the SMILE sequence to Chem object

        # Atoms feature vector
        x = np.array([atom.GetAtomicNum() for atom in mol.GetAtoms()]) # evaluated with atom numbers[1D]

        # Edge index matrix
        A = Chem.rdmolops.GetAdjacencyMatrix(mol) # adjecency matrix
        edge_index = np.argwhere(A != 0).T # (2 X edge_num) [2D]

        data_x.append(x)
        data_edge_index.append(edge_index)
    
    return data_x, data_edge_index

data_x_35, data_edge_index_35 = extract_graph_data(data_35)
data_x_1, data_edge_index_1 = extract_graph_data(data_1)

In [52]:
encoder = OneHotEncoder()

# fitting the encoder on the entire data features
encoder.fit(np.concatenate(data_x_35 + data_x_1).reshape(-1,1))
encoder.categories_

[array([ 5,  6,  7,  8,  9, 14, 15, 16, 17, 35, 53])]

In [55]:
# example
print(data_x_35[0].reshape(-1,1))
print(encoder.transform(data_x_35[0].reshape(-1,1)).toarray())

[[17]
 [ 6]
 [ 6]
 [ 6]
 [ 8]
 [ 6]
 [ 6]
 [ 6]
 [ 6]
 [ 6]
 [16]
 [ 7]]
[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [56]:
data_x_hot_1 = [encoder.transform(x.reshape(-1, 1)).toarray() for x in data_x_1]
data_x_hot_35 = [encoder.transform(x.reshape(-1, 1)).toarray() for x in data_x_35]